In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from scipy.stats import gaussian_kde
import numpy as np

%matplotlib qt

class CelestialBody:
    """
    Represents a celestial body (e.g., moon or test particle) with attributes such as name, color, position, and trail options.

    Attributes:
        name (str): Name of the celestial body.
        color (str): Color used for plotting the body.
        pos (ndarray): 3D positional data (timesteps x 3).
        trail (bool): Whether the body should have a trailing line when animated.
    """
    def __init__(self, name, color, pos, trail=False):
        self.name = name
        self.color = color
        self.pos = pos  # Positional data as a 3D NumPy array (timesteps x 3)
        self.trail = trail


class Dataset:
    """
    Handles the simulation dataset, including reading the header, extracting positional data,
    and initializing celestial bodies for visualization.

    Attributes:
        filepath (str): Path to the binary simulation data file.
        header (dict): Header information extracted from the file.
        num_moons (int): Number of moons in the dataset.
        num_test_particles (int): Number of test particles in the dataset.
        positions (ndarray): 3D array of positions and velocities for all objects (timesteps x objects x 6).
        moons (list): List of CelestialBody objects representing moons.
        test_particles (list): List of CelestialBody objects representing test particles.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.header = self._read_header()
        self.num_moons = int(self.header['Moon Count'])
        self.num_test_particles = int(self.header['Number of Test Particles'])
        self.positions = self._read_binary_file()
        self.moons, self.test_particles = self._initialize_bodies()

    def _read_header(self):
        """
        Reads and parses the header section of the binary file.

        Returns:
            dict: Parsed header as a dictionary.
        """
        header = {}
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                key, value = line.decode('utf-8').strip().split(':', 1)
                header[key.strip()] = self._convert_value(value.strip())
        return header

    def _convert_value(self, value):
        """
        Converts header values to their appropriate data types.

        Args:
            value (str): Header value to be converted.

        Returns:
            int, float, or str: Converted value.
        """
        try:
            if '.' in value:
                return float(value)
            return int(value)
        except ValueError:
            return value

    def _read_binary_file(self):
        """
        Reads the binary data section of the file, skipping the header.

        Returns:
            ndarray: Reshaped positional and velocity data (timesteps x objects x 6).
        """
        length_header = len(self._read_header_raw()) + len("End of Header\n")
        data = np.fromfile(self.filepath, dtype=np.float64, offset=length_header)
        return data.reshape(-1, self.num_moons + self.num_test_particles, 6)

    def _read_header_raw(self):
        """
        Reads the raw header for calculating its length.

        Returns:
            bytes: Raw header data.
        """
        header = []
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                header.append(line)
        return b"".join(header)

    def _initialize_bodies(self):
        """
        Initializes celestial bodies (moons and test particles) with positional data.

        Returns:
            tuple: A list of moon CelestialBody objects and test particle CelestialBody objects.
        """
        moon_names = self.header['Moon Names'].split(', ')
        body_colors = ['yellow', 'red', 'chartreuse', 'lightblue', 'orange', 'brown', 'blue', 'pink', 'red', 'black',
                       'green', 'purple', 'cyan', 'magenta', 'gold', 'silver', 'lime', 'navy', 'maroon', 'crimson']
        moon_colors = body_colors[:self.num_moons]

        moons = [
            CelestialBody(moon_names[i], moon_colors[i], self.positions[:, i, :3], trail=True)
            for i in range(self.num_moons)
        ]

        test_particle_positions = self.positions[:, self.num_moons:, :3]
        test_particles = [
            CelestialBody(str(i), "navy", test_particle_positions[:, i, :], trail=False)
            for i in range(test_particle_positions.shape[1])
        ]

        return moons, test_particles




In [148]:
class AnimationManager:
    def __init__(self, dataset):
        self.dataset = dataset

    def plot_2d(self, coords=[0, 1], n_farthest_filter=10, big_traillength=100, small_traillength=3, frame_time=10,interval=1):
        """
        Animates the simulation, displaying the positions and trails of celestial bodies.

        Args:
            coords (list): Coordinate indices to plot (e.g., [0, 1] for X-Y plane).
            n_farthest_filter (int): Number of moons to exclude from zoomed view based on distance.
            big_traillength (int): Length of the main trail for moons.
            small_traillength (int): Length of the zoomed trail for moons.
            interval (int): Time interval between animation frames.
        """
        # Create a figure with two subplots
        figure, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Set axis limits for the full view
        xmax, xmin = np.max([np.max(body.pos[:, coords[0]]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, coords[0]]) for body in self.dataset.moons])
        ymax, ymin = np.max([np.max(body.pos[:, coords[1]]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, coords[1]]) for body in self.dataset.moons])
        ax.set_xlim(xmin - 0.1 * abs(xmin), xmax + 0.1 * abs(xmax))
        ax.set_ylim(ymin - 0.1 * abs(ymin), ymax + 0.1 * abs(ymax))
        ax.set_aspect('equal', adjustable='box')

        # Filter moons for the zoomed-in view
        nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2) for body in self.dataset.moons])[:-n_farthest_filter]
        filtered_moons = [self.dataset.moons[i] for i in nth_largest_indices]
        ax2.set_xlim(-3e8, 3e8)
        ax2.set_ylim(-3e8, 3e8)
        ax2.set_aspect('equal', adjustable='box')

        # Initialize plot objects for moons and test particles
        moon_lines = {body: ax.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in self.dataset.moons}
        moon_markers = {}
        zoom_lines = {}
        zoom_markers = {}

        # Special marker size for Saturn
        large_marker_size = 15
        default_marker_size = 6

        for body in self.dataset.moons:
            marker_size = large_marker_size if body.name.lower() == "saturn" else default_marker_size
            moon_markers[body] = ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]
            if body in filtered_moons:
                zoom_lines[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0]
                zoom_markers[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]

        test_particle_line = ax2.plot([], [], ".", label="Test Particles", color="navy", markersize=1)[0]

        # Initialization function for the animation
        def init():
            for line in moon_lines.values():
                line.set_data([], [])
            for marker in moon_markers.values():
                marker.set_data([], [])
            for zoom_line in zoom_lines.values():
                zoom_line.set_data([], [])
            for zoom_marker in zoom_markers.values():
                zoom_marker.set_data([], [])
            test_particle_line.set_data([], [])
            return list(moon_lines.values()) + list(moon_markers.values()) + list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]

        # Update function for each frame
        def update(i):
            for body in self.dataset.moons:
                moon_lines[body].set_data(
                    body.pos[max(i - big_traillength, 0):i + 1, coords[0]],  # Include current position
                    body.pos[max(i - big_traillength, 0):i + 1, coords[1]]
                )
                moon_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            for body in filtered_moons:
                zoom_lines[body].set_data(
                    body.pos[max(i - small_traillength, 0):i + 1, coords[0]],  # Include current position
                    body.pos[max(i - small_traillength, 0):i + 1, coords[1]]
                )
                zoom_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            test_particle_line.set_data(
                self.dataset.positions[i, self.dataset.num_moons:, coords[0]],
                self.dataset.positions[i, self.dataset.num_moons:, coords[1]]
            )
            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]


        # Create the animation and assign it to an attribute to prevent garbage collection
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set axis labels
        labels = {0: 'X (m)', 1: 'Y (m)', 2: 'Z (m)'}
        ax.set_xlabel(labels[coords[0]])
        ax.set_ylabel(labels[coords[1]])
        ax2.set_xlabel(labels[coords[0]])
        ax2.set_ylabel(labels[coords[1]])

        # Add a legend
        marker_handles = [moon_markers[body] for body in self.dataset.moons]
        figure.legend(handles=marker_handles, loc='center right', bbox_to_anchor=(0.98, 0.5), title="Moons")

        # Adjust layout for better readability
        plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0.15)

        plt.show()


    def plot_3d(self, n_farthest_filter=10, big_traillength=100, small_traillength=3, frame_time=100, interval=1):
        figure = plt.figure(figsize=(14, 7))

        # Create two 3D subplots
        ax = figure.add_subplot(121, projection='3d')
        ax2 = figure.add_subplot(122, projection='3d')

        # Set wide view axis limits
        xmax, xmin = np.max([np.max(body.pos[:, 0]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 0]) for body in self.dataset.moons])
        ymax, ymin = np.max([np.max(body.pos[:, 1]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 1]) for body in self.dataset.moons])
        zmax, zmin = np.max([np.max(body.pos[:, 2]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 2]) for body in self.dataset.moons])

        # Add debugging prints for axis limits
        # print(f"X Range: {xmin} to {xmax}")
        # print(f"Y Range: {ymin} to {ymax}")
        # print(f"Z Range: {zmin} to {zmax}")

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_zlim(zmin, zmax)

        # Zoomed-in view filter based on distance
        nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2 + body.pos[:, 2]**2) for body in self.dataset.moons])[:-n_farthest_filter]
        filtered_moons = [self.dataset.moons[i] for i in nth_largest_indices]

        # Zoomed-in axis limits
        zoom_range = 3e8
        ax2.set_xlim(-zoom_range, zoom_range)
        ax2.set_ylim(-zoom_range, zoom_range)
        ax2.set_zlim(-zoom_range, zoom_range)

        # Initialize moon plot objects
        moon_lines = {body: ax.plot([], [], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in self.dataset.moons}
        moon_markers = {body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', linestyle='')[0] for body in self.dataset.moons}

        zoom_lines = {body: ax2.plot([], [], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in filtered_moons}
        zoom_markers = {body: ax2.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', linestyle='')[0] for body in filtered_moons}

        # Initialize test particle plot objects
        test_particle_line = ax2.plot([], [], [], ".", label="Test Particles", color="navy", markersize=1)[0]

        def init():
            # Initialize moons
            for line in moon_lines.values():
                line.set_data_3d([], [], [])
            for marker in moon_markers.values():
                marker.set_data_3d([], [], [])
            for line in zoom_lines.values():
                line.set_data_3d([], [], [])
            for marker in zoom_markers.values():
                marker.set_data_3d([], [], [])

            test_particle_line.set_data_3d([], [], [])

            # Add debugging prints for init function
            #print("Init function called.")

            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + \
                [test_particle_line]

        def update(i):
            # Update moons in wide view
            for body in self.dataset.moons:
                moon_lines[body].set_data_3d(
                    body.pos[max(i - big_traillength, 0):i + 1, 0],  # Include current position
                    body.pos[max(i - big_traillength, 0):i + 1, 1],
                    body.pos[max(i - big_traillength, 0):i + 1, 2]
                )
                moon_markers[body].set_data_3d(
                    body.pos[i, 0],
                    body.pos[i, 1],
                    body.pos[i, 2]
                )

            # Update moons in zoomed-in view
            for body in filtered_moons:
                zoom_lines[body].set_data_3d(
                    body.pos[max(i - small_traillength, 0):i + 1, 0],  # Include current position
                    body.pos[max(i - small_traillength, 0):i + 1, 1],
                    body.pos[max(i - small_traillength, 0):i + 1, 2]
                )
                zoom_markers[body].set_data_3d(
                    body.pos[i, 0],
                    body.pos[i, 1],
                    body.pos[i, 2]
                )

            # Update test particles
            test_particle_line.set_data_3d(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1],
                self.dataset.positions[i, self.dataset.num_moons:, 2]
            )

            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + \
                [test_particle_line]


        # Create animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.moons[0].pos.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
        ax2.set_xlabel('X (m)')
        ax2.set_ylabel('Y (m)')
        ax2.set_zlabel('Z (m)')

        handles = [moon_markers[body] for body in self.dataset.moons]
        figure.legend(handles=handles, loc='center right', bbox_to_anchor=(0.98, 0.5), title="Moons")
        plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0.2)

        plt.show()


    def plot_centered(self, moon_name, width, frame_time=10, trail_length=100, target_trail_length=150, interval=1):
        """Single-axis plot centered on a specific moon with no trails for test particles"""
        figure, ax = plt.subplots(figsize=(14, 7))
        target_moon = next((body for body in self.dataset.moons if body.name.lower() == moon_name.lower()), None)
        if not target_moon:
            raise ValueError(f"Moon '{moon_name}' not found!")

        ax.set_xlim(-width / 2, width / 2)
        ax.set_ylim(-width / 2, width / 2)
        ax.set_aspect('equal', adjustable='box')

        # Create a larger marker for the target moon (to make it more visible)
        moon_marker = ax.plot([], [], label=f"{target_moon.name}", color=target_moon.color, marker='o', markersize=8, linestyle='', zorder=5)[0]
        
        # Initialize trail for target moon
        target_trail, = ax.plot([], [], color=target_moon.color, linestyle='-', alpha=0.5, zorder=2)

        # Create markers and trails for other moons
        other_markers = {
            body: ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='', zorder=3)[0]
            for body in self.dataset.moons if body != target_moon
        }
        
        other_trails = {
            body: ax.plot([], [], color=body.color, linestyle='-', alpha=0.5, zorder=1)[0]
            for body in self.dataset.moons if body != target_moon
        }

        # Test particle marker without trails
        test_particle_marker = ax.plot([], [], ".", label="Test Particles", color="navy", markersize=1, zorder=0)[0]

        # Initialize the trail for the centered moon (target_moon)
        target_moon_trail = ax.plot([], [], color='red', linestyle='--', alpha=0.5, zorder=2)[0]

        def init():
            moon_marker.set_data([], [])
            target_trail.set_data([], [])
            target_moon_trail.set_data([], [])
            for marker in other_markers.values():
                marker.set_data([], [])
            for trail in other_trails.values():
                trail.set_data([], [])
            test_particle_marker.set_data([], [])
            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail, target_moon_trail]

        def update(i):
            # Update position for target moon marker
            center_x, center_y = target_moon.pos[i, 0], target_moon.pos[i, 1]
            ax.set_xlim(center_x - width / 2, center_x + width / 2)
            ax.set_ylim(center_y - width / 2, center_y + width / 2)
            moon_marker.set_data(center_x, center_y)

            # Update trail for the target moon
            trail_x = target_moon.pos[max(0, i - trail_length):i + 1, 0]
            trail_y = target_moon.pos[max(0, i - trail_length):i + 1, 1]
            target_trail.set_data(trail_x, trail_y)

            # Update trail for the centered target moon
            target_moon_trail_x = target_moon.pos[max(0, i - target_trail_length):i + 1, 0]
            target_moon_trail_y = target_moon.pos[max(0, i - target_trail_length):i + 1, 1]
            target_moon_trail.set_data(target_moon_trail_x, target_moon_trail_y)

            # Update positions and trails for the other moons
            for body, marker in other_markers.items():
                marker.set_data(body.pos[i, 0], body.pos[i, 1])
                trail_x = body.pos[max(0, i - trail_length):i + 1, 0]
                trail_y = body.pos[max(0, i - trail_length):i + 1, 1]
                other_trails[body].set_data(trail_x, trail_y)

            # Update test particles (positions only, no trails)
            test_particle_marker.set_data(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1]
            )

            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail, target_moon_trail]

        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')

        # Add a customized legend with the markers outside the plot
        marker_handles = [moon_marker] + list(other_markers.values()) + [test_particle_marker]
        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.7, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()

    def plot_centered_3d(self, moon_name, width, frame_time=10, trail_length=100, target_trail_length=150, interval=1):
        """3D plot centered on a specific moon with no trails for test particles"""
        figure = plt.figure(figsize=(14, 7))
        ax = figure.add_subplot(111, projection='3d')

        # Find the target moon
        target_moon = next((body for body in self.dataset.moons if body.name.lower() == moon_name.lower()), None)
        if not target_moon:
            raise ValueError(f"Moon '{moon_name}' not found!")

        # Set initial limits for the 3D view
        ax.set_xlim(-width / 2, width / 2)
        ax.set_ylim(-width / 2, width / 2)
        ax.set_zlim(-width / 2, width / 2)

        # Create a larger marker for the target moon
        moon_marker = ax.plot([], [], [], label=f"{target_moon.name}", color=target_moon.color, marker='o', markersize=8, linestyle='', zorder=5)[0]

        # Initialize trail for the target moon
        target_trail, = ax.plot([], [], [], color=target_moon.color, linestyle='-', alpha=0.5, zorder=2)

        # Create markers and trails for other moons
        other_markers = {
            body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='', zorder=3)[0]
            for body in self.dataset.moons if body != target_moon
        }

        other_trails = {
            body: ax.plot([], [], [], color=body.color, linestyle='-', alpha=0.5, zorder=1)[0]
            for body in self.dataset.moons if body != target_moon
        }

        # Test particle marker without trails
        test_particle_marker = ax.plot([], [], [], ".", label="Test Particles", color="navy", markersize=1, zorder=0)[0]

        def init():
            moon_marker.set_data_3d([], [], [])
            target_trail.set_data_3d([], [], [])
            for marker in other_markers.values():
                marker.set_data_3d([], [], [])
            for trail in other_trails.values():
                trail.set_data_3d([], [], [])
            test_particle_marker.set_data_3d([], [], [])
            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail]

        def update(i):
            # Center the view on the target moon
            center_x, center_y, center_z = target_moon.pos[i, 0], target_moon.pos[i, 1], target_moon.pos[i, 2]
            ax.set_xlim(center_x - width / 2, center_x + width / 2)
            ax.set_ylim(center_y - width / 2, center_y + width / 2)
            ax.set_zlim(center_z - width / 2, center_z + width / 2)

            # Update position for target moon marker
            moon_marker.set_data_3d(center_x, center_y, center_z)

            # Update trail for the target moon
            trail_x = target_moon.pos[max(0, i - trail_length):i + 1, 0]
            trail_y = target_moon.pos[max(0, i - trail_length):i + 1, 1]
            trail_z = target_moon.pos[max(0, i - trail_length):i + 1, 2]
            target_trail.set_data_3d(trail_x, trail_y, trail_z)

            # Update positions and trails for the other moons
            for body, marker in other_markers.items():
                marker.set_data_3d(body.pos[i, 0], body.pos[i, 1], body.pos[i, 2])
                trail_x = body.pos[max(0, i - trail_length):i + 1, 0]
                trail_y = body.pos[max(0, i - trail_length):i + 1, 1]
                trail_z = body.pos[max(0, i - trail_length):i + 1, 2]
                other_trails[body].set_data_3d(trail_x, trail_y, trail_z)

            # Update test particles (positions only, no trails)
            test_particle_marker.set_data_3d(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1],
                self.dataset.positions[i, self.dataset.num_moons:, 2]
            )

            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail]

        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Add a customized legend with the markers outside the plot
        marker_handles = [moon_marker] + list(other_markers.values()) + [test_particle_marker]
        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.75, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()



    def plot_polar_cartesian(self, r_max, r_min=0, frame_time=10, trail_length=100, interval=1, heatmap=False):
        """Polar to Cartesian plot with (r, theta) coordinates for all moons, Saturn, and test particles, with an optional heatmap for test particles."""
        figure, ax = plt.subplots(figsize=(14, 7))
        
        # Set up the axis: r is on the x-axis, theta (0 to 2pi) is on the y-axis
        ax.set_xlim(r_min, r_max)
        ax.set_ylim(0, 2 * np.pi)
        ax.set_xlabel('Radial Distance (r) [m]')
        ax.set_ylabel('Angle (theta) [radians]')
        
        # Helper function to convert Cartesian (x, y) to polar (r, theta)
        def cartesian_to_polar(x, y):
            r = np.sqrt(x ** 2 + y ** 2)
            theta = np.arctan2(y, x)
            return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

        # Create markers and trails for all moons, including Saturn at the origin
        moon_markers = {
            body: ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='')[0]
            for body in self.dataset.moons
        }
        
        moon_trails = {
            body: ax.plot([], [], color=body.color, linestyle='-', alpha=0.5)[0]
            for body in self.dataset.moons
        }

        # Test particles (either as individual markers or as a heatmap)
        if not heatmap:
            test_particle_marker = ax.plot([], [], ".", label="Test Particles", color="navy", markersize=1)[0]
        else:
            # Create an empty 2D histogram for the heatmap (r and theta grid)
            heatmap_data, xedges, yedges = np.histogram2d([], [], bins=(100, 100), range=[[r_min, r_max], [0, 2 * np.pi]])
            heatmap_img = ax.imshow(heatmap_data.T, extent=[r_min, r_max, 0, 2 * np.pi], origin='lower', aspect='auto', cmap='viridis', alpha=0.8)

        # Saturn at origin (barycenter) with an "X" marker
        saturn_marker = ax.plot([0], [0], "x", label="Saturn (Barycenter)", color="yellow", markersize=10)[0]

        def init():
            # Initialize markers and trails to empty
            for marker in moon_markers.values():
                marker.set_data([], [])
            for trail in moon_trails.values():
                trail.set_data([], [])
            if not heatmap:
                test_particle_marker.set_data([], [])
                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                heatmap_img.set_data(np.zeros_like(heatmap_img.get_array()))
                return list(moon_markers.values()) + list(moon_trails.values()) + [heatmap_img, saturn_marker]

        def update(i):
            # Update the positions and trails for all moons in polar coordinates
            for body, marker in moon_markers.items():
                r, theta = cartesian_to_polar(body.pos[i, 0], body.pos[i, 1])
                marker.set_data(r, theta)
                
                # Update the trail of the moons in polar coordinates
                trail_r, trail_theta = cartesian_to_polar(
                    body.pos[max(0, i - trail_length):i + 1, 0],
                    body.pos[max(0, i - trail_length):i + 1, 1]
                )
                moon_trails[body].set_data(trail_r, trail_theta)
            
            if not heatmap:
                # Update the test particles in polar coordinates
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                test_particle_marker.set_data(r_test, theta_test)

                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            
            else:
                # Update the heatmap for test particles
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                
                # Recreate the 2D histogram with current test particle positions
                heatmap_data, xedges, yedges = np.histogram2d(r_test, theta_test, bins=(100, 100), range=[[r_min, r_max], [0, 2 * np.pi]])
                
                # Update the heatmap image data
                heatmap_img.set_data(heatmap_data.T)
                heatmap_img.set_clim(vmin=0, vmax=np.max(heatmap_data))  # Adjust color limits to avoid full saturation

                return list(moon_markers.values()) + list(moon_trails.values()) + [heatmap_img, saturn_marker]

        # Create the animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Add a customized legend with the markers outside the plot
        if not heatmap:
            marker_handles = list(moon_markers.values()) + [test_particle_marker, saturn_marker]
        else:
            marker_handles = list(moon_markers.values()) + [saturn_marker]

        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.85, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()
    
    def plot_polar_cartesian_with_z(self, r_max, r_min=0, z_min=None, z_max=None,elevation = 5 , azimuth = 90 ,frame_time=10, trail_length=100, interval=1, heatmap=False):
        """3D Polar to Cartesian plot with (r, theta, z) coordinates for all moons, Saturn, and test particles."""
        figure = plt.figure(figsize=(14, 7))
        ax = figure.add_subplot(111, projection='3d')  # Create a 3D subplot
        
        # Set the viewing angle (elevation and azimuth)
        ax.view_init(elev=elevation, azim=azimuth) 
    
        # Set up the axis: r is on the x-axis, theta (0 to 3pi) is on the y-axis, and z is on the z-axis
        ax.set_xlim(r_min, r_max)
        ax.set_ylim(-2*np.pi, 2 * np.pi)  # Extend theta axis from -2pi to 2pi
        if z_min is not None and z_max is not None:
            ax.set_zlim(z_min, z_max)  # Set z limits based on user input
        else:
            ax.set_zlim(-r_max, r_max)  # Default z limits based on r_max
        ax.set_xlabel('Radial Distance (r) [m]')
        ax.set_ylabel('Angle (theta) [radians]')
        ax.set_zlabel('Height (z) [m]')
        
        # Helper function to convert Cartesian (x, y) to polar (r, theta)
        def cartesian_to_polar(x, y):
            r = np.sqrt(x ** 2 + y ** 2)
            theta = np.arctan2(y, x)
            return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

        # Create markers and trails for all moons, including Saturn at the origin
        moon_markers = {
            body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='')[0]
            for body in self.dataset.moons
        }
        
        moon_trails = {
            body: ax.plot([], [], [], color=body.color, linestyle='-', alpha=0.5)[0]
            for body in self.dataset.moons
        }

        # Test particles (either as individual markers or as a heatmap)
        if not heatmap:
            test_particle_marker = ax.plot([], [], [], ".", label="Test Particles", color="navy", markersize=.2)[0]
        else:
            # We cannot use a 3D heatmap easily, so let's leave this part out for simplicity
            pass

        # Saturn at origin (barycenter) with an "X" marker
        saturn_marker = ax.plot([0], [0], [0], "x", label="Saturn (Barycenter)", color="yellow", markersize=10)[0]

        def init():
            # Initialize markers and trails to empty
            for marker in moon_markers.values():
                marker.set_data_3d([], [], [])
            for trail in moon_trails.values():
                trail.set_data_3d([], [], [])
            if not heatmap:
                test_particle_marker.set_data_3d([], [], [])
                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                return list(moon_markers.values()) + list(moon_trails.values()) + [saturn_marker]

        def update(i):
            # Update the positions and trails for all moons in polar coordinates
            for body, marker in moon_markers.items():
                r, theta = cartesian_to_polar(body.pos[i, 0], body.pos[i, 1])
                z = body.pos[i, 2]  # Take the z-direction (height) from the third dimension

                # Plot only points within the theta range [0, 2pi]
                if theta <= 2 * np.pi:
                    marker.set_data_3d([r], [theta], [z])
                
                # Update the trail of the moons in polar coordinates
                trail_r, trail_theta = cartesian_to_polar(
                    body.pos[max(0, i - trail_length):i + 1, 0],
                    body.pos[max(0, i - trail_length):i + 1, 1]
                )
                trail_z = body.pos[max(0, i - trail_length):i + 1, 2]

                # Plot only trails within the theta range [0, 2pi]
                valid_idx = trail_theta <= 2 * np.pi
                moon_trails[body].set_data_3d(trail_r[valid_idx], trail_theta[valid_idx], trail_z[valid_idx])
            
            if not heatmap:
                # Update the test particles in polar coordinates
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                z_test = self.dataset.positions[i, self.dataset.num_moons:, 2]  # Test particles in z-direction

                # Plot only points within the theta range [0, 2pi]
                valid_test_idx = theta_test <= 2 * np.pi
                test_particle_marker.set_data_3d(r_test[valid_test_idx], theta_test[valid_test_idx], z_test[valid_test_idx])

                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                return list(moon_markers.values()) + list(moon_trails.values()) + [saturn_marker]

        # Create the animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Add a customized legend with the markers outside the plot
        if not heatmap:
            marker_handles = list(moon_markers.values()) + [test_particle_marker, saturn_marker]
        else:
            marker_handles = list(moon_markers.values()) + [saturn_marker]

        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.85, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()



# Example usage
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-06_17-14-48.bin' # good for 2d and 3d
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_16-54-17.bin'# good for centered plots #small: 2025-01-11_00-13-15, big: 2025-01-11_16-54-17
filepath = 'simulation 2025-01-11_16-54-17.bin'
dataset = Dataset(filepath)
anim_manager = AnimationManager(dataset)

# 2D Animation
#anim_manager.plot_2d()

# 3D Animation
# anim_manager.plot_3d()

# Centered Animation
# For a plot centered around "Daphnis" with a square bounds of 1e8 meters
# anim_manager.plot_centered(moon_name="Daphnis", width=1e8, trail_length=100, interval=1) # To not get dissy one best uses something with low dt and 1 save modularity

# 3D version of Centered
# anim_manager.plot_centered_3d(moon_name="Daphnis", width=4e7, trail_length=100, interval=1) # To not get dissy one best uses something with low dt and 1 save modularity

# Polar plot
# anim_manager.plot_polar_cartesian(r_max =1.5e8, r_min = 1e8, frame_time=10, trail_length=100, interval=1, heatmap=False)

# Cylindrical plot
anim_manager.plot_polar_cartesian_with_z(r_max =1.5e8, r_min = 1.2e8, z_max=1e4 , z_min=-1e4,elevation=8,azimuth=90, frame_time=10, trail_length=100, interval=1)

---------------------------------------------------------------------------------------------------------
File Name: simulation 2025-01-06_17-14-48.bin 
Creation Date: 2025-01-06 17:14:48
Moon Names: Saturn, Mimas, Enceladus, Tethys, Dione, Rhea, Titan, Hyperion, Iapetus, Phoebe, Janus, Epimetheus, Helene, Telesto, Calypso, Atlas, Prometheus, Pandora, Pan, Daphnis
Moon Count: 20
Initial Data Folder: initial_data_moon_count_20_start_date_2017-11-18_00-00-00-000_creation_date_2024-12-24_21-27-40
Epoch: [2458075.5]
dt: 600.0
Timesteps: 10000
Number of Test Particles: 10000
Saved Points Modularity: 20
Skipped Timesteps: 0
Inner Radius: 70000000.0
Outer Radius: 140000000.0
Include Particle-Moon Collisions: True
---------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------
File Name: simulation 2025-01-11_16-54-17.bin 
Creation Date: 2025-01-11 16:54:17
Moon Names: Saturn, Mimas, Enceladus, Tethys, Dione, Rhea, Titan, Hyperion, Iapetus, Phoebe, Janus, Epimetheus, Helene, Telesto, Calypso, Atlas, Prometheus, Pandora, Pan, Daphnis
Moon Count: 20
Initial Data Folder: initial_data_moon_count_20_start_date_2017-11-18_00-00-00-000_creation_date_2024-12-24_21-27-40
Epoch: [2458075.5]
dt: 600.0
Timesteps: 4000
Number of Test Particles: 100000
Saved Points Modularity: 1
Skipped Timesteps: 3000
Inner Radius: 70000000.0
Outer Radius: 140000000.0
Ring Folder: ring_data_2025-01-09
Include Particle-Moon Collisions: True
Numerical Integrator: Leapfrog
---------------------------------------------------------------------------------------------------------

## orbital adjustments

In [39]:
import numpy as np
import ray
G = 6.674 * 10**-11  # m^3 kg^-1 s^-2
M = 5.6834 * 10**26  # kg (mass of the primary body, e.g., planet or star)
class CelestialBody:
    """
    Represents a celestial body (e.g., moon or test particle) with attributes such as name, color, position, and trail options.
 
    Attributes:
        name (str): Name of the celestial body.
        color (str): Color used for plotting the body.
        pos (ndarray): 3D positional data (timesteps x 3).
        trail (bool): Whether the body should have a trailing line when animated.
        orbital_elements (ndarray): Array of orbital elements for each timestep.
        h_vectors (ndarray): Array of angular momentum vectors for each timestep.
    """
    def __init__(self, name, color, pos,vel=0, trail=False):
        self.name = name
        self.color = color
        self.pos = pos  # Positional data as a 3D NumPy array (timesteps x 3)
        self.vel = vel
        self.trail = trail
        # Initialize orbital elements
        # self.orbital_elements = self.calculate_orbital_elements()
        # self.h_vectors = self.calculate_angular_momentum()

    def calculate_orbital_elements(self):
        """
        Takes a particle and returns its corresponding orbital parameters as values.
        Parameters:
        - moon_data: dictionary with moon indices and names
        Returns:
        - result: dictionary with moon names as keys and orbital parameters as values
        """
        # Placeholder for storing orbital elements over time
        orbital_elements = np.zeros((self.pos.shape[0], 6))  # 6 orbital elements for each timestep
        x = self.pos[:, :]
        v = self.vel[:, :] 
        ###### center SHOULD be saturn is not currently

        # Angular momentum vector
        h = np.cross(x, v,axis=1)
        h_abs = np.sqrt(h[:,0]**2+h[:,1]**2+h[:,2]**2)

        # Node vector
        n = np.cross([0, 0, 1], h)
        n_abs = np.sqrt(n[:,0]**2+n[:,1]**2+n[:,1]**2)

        # Eccentricity vector
        e_vec = np.cross(v, h) / (G * M) - np.transpose(np.transpose(x[:,:])/np.sqrt(x[:,0]**2+x[:,1]**2+x[:,2]**2))
        e = np.sqrt(e_vec[:,0]**2+e_vec[:,1]**2+e_vec[:,2]**2)

        # Semi-major axis
        a = h_abs**2 / (G * M)

        # Inclination
        i = np.arccos(h[:,2] / h_abs)

        # Longitude of ascending node
        Omega_temp=np.arccos(n[:,0]/n_abs) #longitude of ascending node
        if np.all(n[:,1]>=0):
            Omega=Omega_temp
        elif np.all(n[:,1]<0):
            Omega=2*np.pi-Omega_temp
        else:
            Omega=2*np.pi*(n[:,1]<0)+(1-2*(n[:,1]<0))*Omega_temp # This case should only happen in the rare event that n is very dependent on time

        # Argument of periapsis
        omega_temp=np.arccos(np.einsum("ij,ij->i",n,e_vec)/n_abs/e) #einsum works, but most importantly best I can find, quite fast as well can do 1e4 computations in .4s
        if np.all(e_vec[:,2]>=0):
            omega=omega_temp
        elif np.all(e_vec[:,2]<0):
            omega=2*np.pi-omega_temp
        else:
            omega=2*np.pi*(e_vec[:,2]<0)+(1-2*(e_vec[:,2]<0))*omega_temp # This case should only happen in the rare event that n is very dependent on time

        # Orbital period
        T = 2 * np.pi * np.sqrt(a**3 / (G * M))

        # Store orbital elements for this timestep
        orbital_params = {
            "inclination": np.degrees(i),
            "ascending_node_longitude": np.degrees(Omega),
            "argument_of_periapsis": np.degrees(omega),
            "eccentricity": e,
            "semimajor_axis": a,
            "period": T
        }
        vectors = {
            "angular_momentum_vector": h,
            "ascending_node_vector": n,
            "eccentricity_vector": e_vec
        }
        return (orbital_params, vectors)

    
    def calculate_angular_momentum(self):
        """
        Calculate the angular momentum vectors for each timestep.
        Returns:
            ndarray: Angular momentum vectors for each timestep.
        """
        # Placeholder for angular momentum vectors over time
        h_vectors = np.zeros((self.pos.shape[0], 3))  # 3 components for each angular momentum vector
        # For each timestep, calculate the angular momentum vector
        for t in range(self.pos.shape[0]):
            x = self.pos[t, :]
            v = self.pos[t, :]  # Assuming position and velocity are the same for now
            # Calculate angular momentum
            h = np.cross(x, v)
            h_vectors[t] = h
        return h_vectors

class Dataset:
    """
    Handles the simulation dataset, including reading the header, extracting positional data,
    and initializing celestial bodies for visualization.

    Attributes:
        filepath (str): Path to the binary simulation data file.
        header (dict): Header information extracted from the file.
        num_moons (int): Number of moons in the dataset.
        num_test_particles (int): Number of test particles in the dataset.
        positions (ndarray): 3D array of positions and velocities for all objects (timesteps x objects x 6).
        moons (list): List of CelestialBody objects representing moons.
        test_particles (list): List of CelestialBody objects representing test particles.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.header = self._read_header()
        self.num_moons = int(self.header['Moon Count'])
        self.num_test_particles = int(self.header['Number of Test Particles'])
        self.positions = self._read_binary_file()
        self.moons, self.test_particles = self._initialize_bodies()

    def _read_header(self):
        """
        Reads and parses the header section of the binary file.

        Returns:
            dict: Parsed header as a dictionary.
        """
        header = {}
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                key, value = line.decode('utf-8').strip().split(':', 1)
                header[key.strip()] = self._convert_value(value.strip())
        return header

    def _convert_value(self, value):
        """
        Converts header values to their appropriate data types.

        Args:
            value (str): Header value to be converted.

        Returns:
            int, float, or str: Converted value.
        """
        try:
            if '.' in value:
                return float(value)
            return int(value)
        except ValueError:
            return value

    def _read_binary_file(self):
        """
        Reads the binary data section of the file, skipping the header.

        Returns:
            ndarray: Reshaped positional and velocity data (timesteps x objects x 6).
        """
        length_header = len(self._read_header_raw()) + len("End of Header\n")
        data = np.fromfile(self.filepath, dtype=np.float64, offset=length_header)
        return data.reshape(-1, self.num_moons + self.num_test_particles, 6)

    def _read_header_raw(self):
        """
        Reads the raw header for calculating its length.

        Returns:
            bytes: Raw header data.
        """
        header = []
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                header.append(line)
        return b"".join(header)

    def _initialize_bodies(self):
        """
        Initializes celestial bodies (moons and test particles) with positional data.

        Returns:
            tuple: A list of moon CelestialBody objects and test particle CelestialBody objects.
        """
        moon_names = self.header['Moon Names'].split(', ')
        body_colors = ['yellow', 'red', 'chartreuse', 'lightblue', 'orange', 'brown', 'blue', 'pink', 'red', 'black',
                       'green', 'purple', 'cyan', 'magenta', 'gold', 'silver', 'lime', 'navy', 'maroon', 'crimson']
        moon_colors = body_colors[:self.num_moons]

        moons = [
            CelestialBody(moon_names[i], moon_colors[i], self.positions[:, i, :3],self.positions[:,i,3:], trail=True)
            for i in range(self.num_moons)
        ]

        test_particle_positions = self.positions[:, self.num_moons:, :3]
        test_particles =    [
            CelestialBody(str(i), "navy", test_particle_positions[:, i, :],self.positions[:,self.num_moons+i,3:], trail=False)
            for i in range(test_particle_positions.shape[1])
        ]

        return moons, test_particles
    def Orbital_elements_of_bodies(self):
        for parts in self.test_particles:
            if parts.pos[-1,0]>1e11: parts.orbital_elements=None; continue
            parts.orbital_elements=parts.calculate_orbital_elements()

# Constants (assuming they're already defined)
filepath = 'simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)


In [22]:
test=dataset.test_particles[2]

### calculating orbital elements

In [30]:
# Get the orbital elements for all timesteps
orbital_elements = test.calculate_orbital_elements()
# Access a specific orbital element (e.g., inclination) at each timestep
inclination = orbital_elements[0]["inclination"]
#The first column represents inclination
#Get the angular momentum vectors 
angular_momentum = test.calculate_angular_momentum()
 

In [34]:
len(dataset.positions[0])

10020

In [41]:
def Orbital_elements_many():
    for parts in dataset.test_particles:
        if parts.pos[-1,0]>1e11: parts.orbital_elements=None; continue
        parts.orbital_elements=parts.calculate_orbital_elements()

Orbital_elements_ring()

NameError: name 'Orbital_elements_ring' is not defined

In [42]:
dataset.Orbital_elements_of_bodies()

In [81]:
ding=[dataset.test_particles[240+i].orbital_elements[1] for i in range(10)]
els=dataset.test_particles[240].orbital_elements[1]


In [56]:
ding

[{'angular_momentum_vector': array([[-1.04270205e+08,  2.60372158e+07,  1.66089737e+12],
         [-9.59295618e+07,  3.45082294e+07,  1.66211104e+12],
         [-8.85692350e+07,  4.54124420e+07,  1.66334530e+12],
         ...,
         [ 1.37813829e+08,  1.92913607e+07,  1.66381980e+12],
         [ 1.52943186e+08,  3.48888426e+07,  1.66492227e+12],
         [ 1.52943186e+08,  3.48888426e+07,  1.66492227e+12]]),
  'ascending_node_vector': array([[-2.60372158e+07, -1.04270205e+08,  0.00000000e+00],
         [-3.45082294e+07, -9.59295618e+07,  0.00000000e+00],
         [-4.54124420e+07, -8.85692350e+07,  0.00000000e+00],
         ...,
         [-1.92913607e+07,  1.37813829e+08,  0.00000000e+00],
         [-3.48888426e+07,  1.52943186e+08,  0.00000000e+00],
         [-3.48888426e+07,  1.52943186e+08,  0.00000000e+00]]),
  'eccentricity_vector': array([[-2.69766533e-03, -7.55937060e-03, -5.08527217e-08],
         [-2.38010121e-03, -5.25013471e-03, -2.83670655e-08],
         [-2.67127465e-03

In [63]:
els["eccentricity_vector"][0,:]

array([-2.69766533e-03, -7.55937060e-03, -5.08527217e-08])

In [79]:
firsts=np.concatenate([vec["eccentricity_vector"][100:101] for vec in ding],axis=0)*2
lasts=np.concatenate([vec["eccentricity_vector"][-1:] for vec in ding],axis=0)*2

In [72]:
lasts.shape

(10, 3)

In [80]:
ax = plt.figure().add_subplot(projection='3d')
evec=els["eccentricity_vector"]
# ax.set_xlim((-2,2))
# ax.set_ylim(-2,2)
# ax.set_zlim(-2,2)
# ax.set_aspect('equal', adjustable='box')
# plt.suptitle("before")
# plt.plot(points[0],points[1],points[2])
plt.quiver(np.zeros(len(firsts)),np.zeros(len(firsts)),np.zeros(len(firsts)),lasts[:,0],firsts[:,1],firsts[:,2],color="black")
plt.quiver(np.zeros(len(lasts)),np.zeros(len(lasts)),np.zeros(len(lasts)),lasts[:,0],lasts[:,1],lasts[:,2],color="g")
# h=np.cross(points,speeds,axis=0)
# plt.quiver(points[0],points[1],points[2],h[0],h[1],h[2],color="r")
plt.show()

In [18]:
orbital_elements[3]

array([0.00544869, 0.00502967, 0.00466544, ..., 0.00306176, 0.00318557,
       0.00318557])

## seperating theta in bins

In [282]:
def cartesian_to_polar(x, y):
    r = np.sqrt(x ** 2 + y ** 2)
    theta = np.arctan2(y, x)
    return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]
bins=20
s=int(np.ceil(np.sqrt(bins)/4))
i=0
r_test, theta_test = cartesian_to_polar(
                    dataset.positions[i, dataset.num_moons:, 0],
                    dataset.positions[i, dataset.num_moons:, 1]
                )
sieve=r_test<1e11
# plt.hist(theta_test,bins=bins)
edges=np.histogram(theta_test,bins=bins)[1]
def split_theta_in_bins(r_test, theta_test,theta_min,theta_max,i):
    bindata_indexes=np.nonzero(np.logical_and(theta_min<theta_test,theta_test<theta_max))[0]
    return r_test[bindata_indexes],dataset.positions[i, dataset.num_moons:, 2][sieve][bindata_indexes]-dataset.positions[i, 0, 2],theta_test[bindata_indexes]
# plt.figure()
figure=plt.figure()
axes=figure.subplots(s,s)
k=0
lines=dict()
rlabel=True

for axlist in axes:
    axlist[0].set_ylabel("z (m)")
    rlabel=not rlabel
    for ax in axlist:
        if  rlabel:
            ax.set_xlabel(f"r (m)")
        if k>=bins: break
        r,z,_=split_theta_in_bins(r_test[sieve], theta_test[sieve],edges[k],edges[k+1],i)
        lines[k]=ax.plot(r,z,"g.",markersize=2)[0]
        # ax.set_xlim(1.32e8,1.42e8)
        # ax.set_ylim(-11e3,-3e3)
        k+=4

plt.show()
def update(i):
    k=0
    for axlist in axes:
        for ax in axlist:
            if k>=bins: break
            r_test, theta_test = cartesian_to_polar(
                        dataset.positions[i, dataset.num_moons:, 0]-dataset.positions[i, 0, 0],
                        dataset.positions[i, dataset.num_moons:, 1]-dataset.positions[i, 0, 1]
                    )
            r,z,_=split_theta_in_bins(r_test[sieve], theta_test[sieve],edges[k],edges[k+1],i)
            lines[k].set_data(r,z)
            k+=4
    # figure.suptitle(f"{i}")

animation=anim.FuncAnimation(figure,update,
                   frames=np.arange(0, dataset.positions.shape[0], 1),
                   interval=1)
# writergif = anim.PillowWriter(fps=15)
# animation.save('lussen.gif',writer=writergif)

In [194]:
k=0
r,z,thet=split_theta_in_bins(r_test[sieve], theta_test[sieve],edges[k],edges[k+1],i)
ax.plot(r,z,".",markersize=3)
thet   
figure = plt.figure(figsize=(14, 7))

# Create two 3D subplots
ax = figure.add_subplot(121, projection='3d')
# ax.set_aspect()
dots=ax.plot(r, thet, z, ".", label="Test Particles", color="navy", markersize=.5)[0]
ax.set_zlim(-2e4,0)
ax.set_xlim(.7e8,2.4e8)
def update(i):
    r_test, theta_test = cartesian_to_polar(
                    dataset.positions[i, dataset.num_moons:, 0],
                    dataset.positions[i, dataset.num_moons:, 1]
                )
    r,z,thet=split_theta_in_bins(r_test[sieve], theta_test[sieve],edges[k],edges[k+1],i)
    dots.set_data_3d(r,thet,z)
anim.FuncAnimation(figure,update,
                   frames=np.arange(0, dataset.positions.shape[0], 1),
                   interval=1)

In [120]:
np.nonzero((r_test<1e11) ^ (dataset.positions[i, dataset.num_moons:, 2]<1e11))

(array([ 615, 1079, 1195, 1314, 1409, 1441, 1611, 1676, 1689, 1768, 2231,
        2543, 2626, 2653, 2671, 2693, 2817, 3077, 3833, 4184, 4245, 4389,
        4395, 4413, 4416, 4844, 5771, 5975, 6243, 6362, 6670, 6889, 7584,
        8082, 8671, 9033, 9266], dtype=int64),)

In [125]:
r_test[615],dataset.positions[i, dataset.num_moons:, 2][615]

(137499298.28140146, 1000000000000.0)

In [127]:
plt.plot(dataset.positions[i, dataset.num_moons:, 2][sieve])

## Fourrier transforms

In [44]:
anim_manager.animation.pause()

In [154]:
r_allparts=np.sqrt(dataset.positions[-1, dataset.num_moons:, 0]**2+dataset.positions[-1, dataset.num_moons:, 1]**2+dataset.positions[-1, dataset.num_moons:, 2]**2)
r_remaining=r_allparts[r_allparts<1e11]
print(r_allparts.shape)
# plt.plot( r_remaining,np.ones(len(r_remaining)),".",markersize=.01)
omega=np.arange(1e-9,1e-6,1e-9)
w,r=np.meshgrid(omega,r_remaining)
transform=np.sum(np.cos(w*r),axis=0)
plt.plot(omega,(transform.real))

(10000,)


In [146]:
r

array([[1.30614664e+08, 1.30614664e+08, 1.30614664e+08, ...,
        1.30614664e+08, 1.30614664e+08, 1.30614664e+08],
       [1.33301056e+08, 1.33301056e+08, 1.33301056e+08, ...,
        1.33301056e+08, 1.33301056e+08, 1.33301056e+08],
       [9.67940732e+07, 9.67940732e+07, 9.67940732e+07, ...,
        9.67940732e+07, 9.67940732e+07, 9.67940732e+07],
       ...,
       [1.19170483e+08, 1.19170483e+08, 1.19170483e+08, ...,
        1.19170483e+08, 1.19170483e+08, 1.19170483e+08],
       [1.07573462e+08, 1.07573462e+08, 1.07573462e+08, ...,
        1.07573462e+08, 1.07573462e+08, 1.07573462e+08],
       [8.96611715e+07, 8.96611715e+07, 8.96611715e+07, ...,
        8.96611715e+07, 8.96611715e+07, 8.96611715e+07]])

In [150]:
r_copy=r_remaining
r_copy.sort()
plt.plot(r_copy,np.arange(0,len(r_copy),1))

In [153]:
plt.hist(r_remaining,bins=40)

(array([ 77., 167., 187., 201., 176., 193., 223., 208., 186., 258., 193.,
        236., 219., 211., 209., 218., 246., 239., 241., 261., 288., 259.,
        289., 225., 248., 277., 346., 221., 317., 288., 272., 341., 288.,
        392., 258., 377., 254., 362., 270., 132.]),
 array([6.94870476e+07, 7.12800761e+07, 7.30731045e+07, 7.48661329e+07,
        7.66591614e+07, 7.84521898e+07, 8.02452183e+07, 8.20382467e+07,
        8.38312751e+07, 8.56243036e+07, 8.74173320e+07, 8.92103604e+07,
        9.10033889e+07, 9.27964173e+07, 9.45894458e+07, 9.63824742e+07,
        9.81755026e+07, 9.99685311e+07, 1.01761560e+08, 1.03554588e+08,
        1.05347616e+08, 1.07140645e+08, 1.08933673e+08, 1.10726702e+08,
        1.12519730e+08, 1.14312759e+08, 1.16105787e+08, 1.17898815e+08,
        1.19691844e+08, 1.21484872e+08, 1.23277901e+08, 1.25070929e+08,
        1.26863958e+08, 1.28656986e+08, 1.30450015e+08, 1.32243043e+08,
        1.34036071e+08, 1.35829100e+08, 1.37622128e+08, 1.39415157e+08,
      

In [59]:
import scipy.special as special

In [77]:
r_allparts=np.sqrt(dataset.positions[-1, dataset.num_moons:, 0]**2+dataset.positions[-1, dataset.num_moons:, 1]**2+dataset.positions[-1, dataset.num_moons:, 2]**2)
r_remaining=r_allparts[r_allparts<1e11]
print(r_allparts.shape)
# plt.plot( r_remaining,np.ones(len(r_remaining)),".",markersize=.01)
omega=np.arange(1e-9,1e-6,1e-9)
w,r=np.meshgrid(omega,r_remaining)
transform=np.sum(special.hankel1(1,w*r),axis=0)
plt.plot(omega,(transform.real))

(10000,)


c:\Users\kdvan\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\eventloops.py:113: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  app.exec_()


In [178]:
np.fft.fftfreq(10, 600)

array([ 0.        ,  0.00016667,  0.00033333,  0.0005    ,  0.00066667,
       -0.00083333, -0.00066667, -0.0005    , -0.00033333, -0.00016667])

In [179]:
np

NameError: name 't' is not defined

In [186]:
np.linspace(0,len(r),600)

array([   0.        ,   16.4490818 ,   32.89816361,   49.34724541,
         65.79632721,   82.24540902,   98.69449082,  115.14357262,
        131.59265442,  148.04173623,  164.49081803,  180.93989983,
        197.38898164,  213.83806344,  230.28714524,  246.73622705,
        263.18530885,  279.63439065,  296.08347245,  312.53255426,
        328.98163606,  345.43071786,  361.87979967,  378.32888147,
        394.77796327,  411.22704508,  427.67612688,  444.12520868,
        460.57429048,  477.02337229,  493.47245409,  509.92153589,
        526.3706177 ,  542.8196995 ,  559.2687813 ,  575.71786311,
        592.16694491,  608.61602671,  625.06510851,  641.51419032,
        657.96327212,  674.41235392,  690.86143573,  707.31051753,
        723.75959933,  740.20868114,  756.65776294,  773.10684474,
        789.55592654,  806.00500835,  822.45409015,  838.90317195,
        855.35225376,  871.80133556,  888.25041736,  904.69949917,
        921.14858097,  937.59766277,  954.04674457,  970.49582

In [192]:
k

NameError: name 'k' is not defined

In [254]:
k_arr=np.arange(1e-9,1e-6,1e-7)
f_arr=np.fft.fftfreq(1000, 600)

In [255]:
Z=np.zeros((len(k_arr),len(f_arr)),complex)
for i in range(len(k_arr)):
    for p in range(len(f_arr)):
        Z[i,p]=np.sum(np.exp(1j*(k_arr[i]*r_remaining+f_arr[p]*np.linspace(0,600,len(r)))))

In [256]:
np.meshgrid([1,2],[5,7,9])

[array([[1, 2],
        [1, 2],
        [1, 2]]),
 array([[5, 5],
        [7, 7],
        [9, 9]])]

In [258]:
f_arr

array([ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06,  5.00000000e-06,
        6.66666667e-06,  8.33333333e-06,  1.00000000e-05,  1.16666667e-05,
        1.33333333e-05,  1.50000000e-05,  1.66666667e-05,  1.83333333e-05,
        2.00000000e-05,  2.16666667e-05,  2.33333333e-05,  2.50000000e-05,
        2.66666667e-05,  2.83333333e-05,  3.00000000e-05,  3.16666667e-05,
        3.33333333e-05,  3.50000000e-05,  3.66666667e-05,  3.83333333e-05,
        4.00000000e-05,  4.16666667e-05,  4.33333333e-05,  4.50000000e-05,
        4.66666667e-05,  4.83333333e-05,  5.00000000e-05,  5.16666667e-05,
        5.33333333e-05,  5.50000000e-05,  5.66666667e-05,  5.83333333e-05,
        6.00000000e-05,  6.16666667e-05,  6.33333333e-05,  6.50000000e-05,
        6.66666667e-05,  6.83333333e-05,  7.00000000e-05,  7.16666667e-05,
        7.33333333e-05,  7.50000000e-05,  7.66666667e-05,  7.83333333e-05,
        8.00000000e-05,  8.16666667e-05,  8.33333333e-05,  8.50000000e-05,
        8.66666667e-05,  

In [267]:
for i in range(len(k_arr)):
    for p in range(len(f_arr)):
        assert k_arr[i]==X_freq[i,p]
        assert f_arr[p]==Y_freq[i,p]

In [263]:
Y_freq

array([[ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06],
       [ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06],
       [ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06],
       ...,
       [ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06],
       [ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06],
       [ 0.00000000e+00,  1.66666667e-06,  3.33333333e-06, ...,
        -5.00000000e-06, -3.33333333e-06, -1.66666667e-06]])

In [283]:
X_freq,Y_freq=np.meshgrid(np.linspace(-1,1,5),np.linspace(-1000,1000,5000),indexing="ij")
Z=X_freq**2+Y_freq**2

fig = plt.figure(figsize=(14, 7))
ax1 = fig.add_subplot( projection='3d')
ax1.plot_surface(X_freq, Y_freq, np.abs(Z), cmap='Reds', edgecolor='none', alpha=0.6)
# ax1.set_ylim3d(f_arr[0],f_arr[-1])
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

Text(0.5, 0, 'Z')

In [262]:
X_freq,Y_freq=np.meshgrid(k_arr,f_arr,indexing="ij")
fig = plt.figure(figsize=(14, 7))
 
ax1 = fig.add_subplot( projection='3d')

ax1.plot_surface(X_freq, Y_freq, np.abs(Z), cmap='Reds', edgecolor='none', alpha=0.6)
ax1.set_ylim3d(f_arr[0],f_arr[-1])
ax1.set_xlabel('k')
ax1.set_ylabel('f')
ax1.set_zlabel('ft')


Text(0.5, 0, 'ft')

In [211]:
Z

array([[9792.75546193+1070.32937098j, 9722.96880616+1557.77647134j,
        9620.9192108 +2039.22372749j, 9487.16481026+2512.27504962j,
        9322.45305873+2974.58786842j, 9651.83178718-1371.32048054j,
        9744.91337481 -890.56143807j, 9805.86754318 -403.5836912j ,
        9834.27577022  +87.19172399j, 9829.91421881 +579.31610315j],
       [1059.22769814 +724.13619044j, 1022.70145749 +772.55305551j,
         983.17704676 +818.44282663j,  940.85322592 +861.59729653j,
         895.944134   +901.82499723j, 1191.16821867 +452.43123924j,
        1171.98713846 +509.87180524j, 1149.08832101 +566.02807022j,
        1122.57559445 +620.62978002j, 1092.57375179 +673.41587636j],
       [-385.02176922 +851.57038605j, -430.09082599 +830.6730761j ,
        -473.64998652 +806.73954036j, -515.47167026 +779.89282457j,
        -555.33897516 +750.27412226j, -145.5364567  +907.56956066j,
        -194.48159677 +903.01399343j, -243.15752811 +895.1031142j ,
        -291.3081212  +883.85971161j, -338.679

In [195]:
r.shape

(9853, 999)

In [204]:
f_arr.shape

(10,)

In [185]:
t_[2].shape

(10, 10, 600)

In [180]:
k_arr=np.arange(1e-9,1e-6,1e-7)
f_arr=np.fft.fftfreq(10, 600)
k_,f_,r_=np.meshgrid(k_arr,f_arr,r)
t_=np.meshgrid(k_arr,f_arr,np.linspace(0,len(r),600))[2]

In [229]:
# x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2, fftshift
 
# Function to create the cos wave for (x, y)
def cos_wave(x, y):
    return np.cos(0.5 * np.pi * x + 1.5 * np.pi * y)
 
# Generate x, y data points
x = np.linspace(0, 70, 50)  # X data
print(x)
y = np.linspace(0, 70, 50) # Y data
X, Y = np.meshgrid(x, y)  # Create a grid of X and Y
Z = cos_wave(X, Y)  # Calculate Z as cos(0.5*pi*x + 1.5*pi*y)
 
# Create the 3D sinusoidal wave plot (Left)
fig = plt.figure(figsize=(14, 7))
 
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_wireframe(X, Y, Z, color='red', alpha=0.6)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title(r'$\cos(0.5\pi x + 1.5\pi y)$')
 
# Perform 2D FFT on the wave and shift zero frequency components to the center
Z_fft = fftshift(fft2(Z))
 
# Get the amplitude of the FFT
amplitude = np.abs(Z_fft)
 
# Create the 3D FFT plot (Right)
ax2 = fig.add_subplot(122, projection='3d')
X_freq, Y_freq = np.meshgrid(np.fft.fftfreq(X.shape[0]), np.fft.fftfreq(Y.shape[1]))
 
ax2.plot_surface(X_freq, Y_freq, amplitude, cmap='Reds', edgecolor='none', alpha=0.6)
ax2.set_xlabel('Frequency (X)')
ax2.set_ylabel('Frequency (Y)')
ax2.set_zlabel('Amplitude')
ax2.set_title(r'FFT[$\cos(0.5\pi x + 1.5\pi y)$]')

[ 0.          1.42857143  2.85714286  4.28571429  5.71428571  7.14285714
  8.57142857 10.         11.42857143 12.85714286 14.28571429 15.71428571
 17.14285714 18.57142857 20.         21.42857143 22.85714286 24.28571429
 25.71428571 27.14285714 28.57142857 30.         31.42857143 32.85714286
 34.28571429 35.71428571 37.14285714 38.57142857 40.         41.42857143
 42.85714286 44.28571429 45.71428571 47.14285714 48.57142857 50.
 51.42857143 52.85714286 54.28571429 55.71428571 57.14285714 58.57142857
 60.         61.42857143 62.85714286 64.28571429 65.71428571 67.14285714
 68.57142857 70.        ]


Text(0.5, 0.92, 'FFT[$\\cos(0.5\\pi x + 1.5\\pi y)$]')

In [228]:
np.meshgrid(np.fft.fftfreq(X.shape[0]), np.fft.fftfreq(Y.shape[1]))[1].shape

(50, 100)

In [81]:
(1/1.26e-7)/1e6

7.936507936507938

In [80]:
(1/4.8e-8)/1e6

20.833333333333332

In [76]:
start=10
plt.plot(omega[start:],np.abs(transform)[start:])
# plt.plot(omega,np.angle(transform)*1e4/6.28,"g")

ValueError: x and y must have same first dimension, but have shapes (890,) and (989,)

In [28]:
r_remaining.shape

(9853,)

In [75]:
omega=np.arange(1e-8,10e-8,1e-10)
np.average(np.meshgrid(omega,r_remaining)[0],axis=0)

array([1.00e-08, 1.01e-08, 1.02e-08, 1.03e-08, 1.04e-08, 1.05e-08,
       1.06e-08, 1.07e-08, 1.08e-08, 1.09e-08, 1.10e-08, 1.11e-08,
       1.12e-08, 1.13e-08, 1.14e-08, 1.15e-08, 1.16e-08, 1.17e-08,
       1.18e-08, 1.19e-08, 1.20e-08, 1.21e-08, 1.22e-08, 1.23e-08,
       1.24e-08, 1.25e-08, 1.26e-08, 1.27e-08, 1.28e-08, 1.29e-08,
       1.30e-08, 1.31e-08, 1.32e-08, 1.33e-08, 1.34e-08, 1.35e-08,
       1.36e-08, 1.37e-08, 1.38e-08, 1.39e-08, 1.40e-08, 1.41e-08,
       1.42e-08, 1.43e-08, 1.44e-08, 1.45e-08, 1.46e-08, 1.47e-08,
       1.48e-08, 1.49e-08, 1.50e-08, 1.51e-08, 1.52e-08, 1.53e-08,
       1.54e-08, 1.55e-08, 1.56e-08, 1.57e-08, 1.58e-08, 1.59e-08,
       1.60e-08, 1.61e-08, 1.62e-08, 1.63e-08, 1.64e-08, 1.65e-08,
       1.66e-08, 1.67e-08, 1.68e-08, 1.69e-08, 1.70e-08, 1.71e-08,
       1.72e-08, 1.73e-08, 1.74e-08, 1.75e-08, 1.76e-08, 1.77e-08,
       1.78e-08, 1.79e-08, 1.80e-08, 1.81e-08, 1.82e-08, 1.83e-08,
       1.84e-08, 1.85e-08, 1.86e-08, 1.87e-08, 1.88e-08, 1.89e

In [92]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_fft(self, data, dt=600):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, dt)  # Frequency bins
        fft_values = np.fft.fft(data)   # FFT of the data
        fft_values = fft_values / len(data)
        return freqs, np.abs(fft_values)  # Only return the magnitude

    def analyze_ring_waveform(self):
        """
        Analyzes the waveforms in the test particles' radial distances using FFT.

        Returns:
            None: Displays a plot of the FFT spectrum.
        """
        # Extract radial distances of all test particles across all time steps
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Compute the FFT for each test particle's radial distance
        plt.figure(figsize=(10, 6))
        for i in range(self.num_test_particles):
            radial_data = radial_distances[:, i]
            freqs, fft_values = self.compute_fft(radial_data)

            # Plot the FFT results for the test particle
            plt.plot(freqs, fft_values, label=f"Test Particle {i + 1}")

        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Test Particles\' Radial Distance')
        #plt.legend() not safe at all
        plt.xlim(0.00000, 1/2/self.dataset.header['dt'])
        print(self.dataset.header['dt'])
        plt.show()

# Example usage
filepath = 'simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the waveform in the ring using FFT
fourier_rings.analyze_ring_waveform()

600.0


In [87]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_fft(self, data, dt=1):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, dt)  # Frequency bins
        fft_values = np.fft.fft(data)   # FFT of the data
        fft_values = fft_values / len(data)
        return freqs[:N // 2], np.abs(fft_values[:N // 2])  # Only return positive frequencies

    def group_particles(self, radial_distances, num_groups=5):
        """
        Groups test particles based on their mean radial distances.

        Args:
            radial_distances (ndarray): Radial distances of all test particles over time.
            num_groups (int): Number of groups to divide particles into.

        Returns:
            dict: A dictionary where keys are group indices and values are particle indices.
        """
        mean_distances = np.mean(radial_distances, axis=0)
        sorted_indices = np.argsort(mean_distances)
        group_size = self.num_test_particles // num_groups
        
        groups = {}
        for group_idx in range(num_groups):
            start_idx = group_idx * group_size
            end_idx = (group_idx + 1) * group_size
            groups[group_idx] = sorted_indices[start_idx:end_idx]
        
        return groups

    def analyze_ring_waveform_by_group(self, num_groups=5):
        """
        Analyzes the waveforms of radial distances for groups of test particles using FFT.

        Args:
            num_groups (int): Number of groups to divide particles into.

        Returns:
            None: Displays a plot of the FFT spectrum for each group.
        """
        # Extract radial distances of all test particles across all time steps
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Group particles based on their mean radial distances
        groups = self.group_particles(radial_distances, num_groups=num_groups)

        plt.figure(figsize=(12, 8))
        for group_idx, particle_indices in groups.items():
            # Compute the mean radial distance for the group
            group_data = np.mean(radial_distances[:, particle_indices], axis=1)
            freqs, fft_values = self.compute_fft(group_data, dt=self.dataset.header['dt'])

            # Plot the FFT for the group
            plt.plot(freqs, fft_values, label=f"Group {group_idx + 1} ({len(particle_indices)} particles)")

        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Test Particles\' Radial Distance by Group')
        plt.legend()
        plt.xlim(0.0000, 1/(2*((self.dataset.header['dt']))))
        plt.show()

# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the waveform in the ring using FFT by groups
fourier_rings.analyze_ring_waveform_by_group(num_groups=5)


FileNotFoundError: [Errno 2] No such file or directory: 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'

In [158]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset, num_bins=1000):
        """
        Initializes the FourierRings class with a given dataset and radial bins.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
            num_bins (int): Number of radial bins to compute the density.
        """
        self.dataset = dataset
        self.num_bins = num_bins
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)

    def compute_fft(self, data, dt=1):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, d=dt)  # Frequency bins
        fft_values = np.fft.fft(data)    # FFT of the data
        fft_values = fft_values / N      # Normalize the FFT amplitude
        return freqs[:N // 2], np.abs(fft_values[:N // 2])  # Return positive frequencies

    def compute_density(self):
        """
        Computes the density profile of test particles in radial bins over time.

        Returns:
            ndarray: Radial density as a 2D array (time_steps x num_bins).
        """
        # Radial bin edges
        max_radius = 1.5  # Adjust based on the system's scale
        min_radius = 0.5  # Adjust based on the system's scale
        bin_edges = np.linspace(min_radius, max_radius, self.num_bins + 1)
        
        # Initialize density array
        density = np.zeros((self.time_steps, self.num_bins))
        
        for t in range(self.time_steps):
            x = self.dataset.positions[t, self.dataset.num_moons:, 0]
            y = self.dataset.positions[t, self.dataset.num_moons:, 1]
            r = self.cartesian_to_polar(x, y)[0]
            
            # Compute histogram for this time step
            density[t, :], _ = np.histogram(r, bins=bin_edges)
        
        return density, bin_edges

    def analyze_density_waveform(self):
        """
        Analyzes the density waveform by computing FFT of density in each radial bin.

        Returns:
            None: Displays a plot of the FFT spectrum for each radial bin.
        """
        # Compute radial density
        density, bin_edges = self.compute_density()
        bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
        
        plt.figure(figsize=(12, 8))
        
        # Compute FFT for each radial bin
        for i in range(self.num_bins):
            density_time_series = density[:, i]
            freqs, fft_values = self.compute_fft(density_time_series, dt=self.dataset.header['dt'])
            
            # Plot the FFT result
            plt.plot(freqs, fft_values, label=f"Radial Bin {i + 1} ({bin_centers[i]:.2f} AU)")
        
        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Density Waveform in Radial Bins')
        plt.legend(loc="upper right", fontsize='small', ncol=2)
        plt.xlim(0, 1/(2*((self.dataset.header['dt']))))
        plt.show()

# Example usage
filepath = 'simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the density waveform using FFT
fourier_rings.analyze_density_waveform()


In [157]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_density(self, num_bins):
        """
        Computes the density in radial bins for each time step.

        Args:
            num_bins (int): Number of radial bins.

        Returns:
            ndarray: 2D array where each row corresponds to density in radial bins at a time step.
        """
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Define radial bins
        r_min, r_max = np.min(radial_distances), np.max(radial_distances)
        radial_bins = np.linspace(r_min, r_max, num_bins + 1)

        # Compute density for each time step
        density = np.zeros((self.time_steps, num_bins))
        for t in range(self.time_steps):
            density[t], _ = np.histogram(radial_distances[t], bins=radial_bins, density=True)

        return density, radial_bins

    def animate_density(self, num_bins=50):
        """
        Animates the density in a polar plot.

        Args:
            num_bins (int): Number of radial bins for density calculation.
        """
        # Compute density
        density, radial_bins = self.compute_density(num_bins)

        # Create theta for polar plot
        theta = np.linspace(0, 2 * np.pi, num_bins, endpoint=False)

        # Set up the polar plot
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        line, = ax.plot([], [], lw=2)
        ax.set_ylim(0, np.max(density))  # Set y-axis limit to maximum density

        def update(frame):
            """
            Update function for animation.
            """
            line.set_data(theta, density[frame])
            return line,

        ani = FuncAnimation(
            fig, update, frames=self.time_steps, interval=100, blit=True
        )

        # Show the animation
        plt.show()

# Example usage
filepath = 'simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Animate the density waveform
fourier_rings.animate_density(num_bins=50)
